# ライブラリの読み込み

In [210]:
import pandas as pd
import jaconv
import re
import emoji

# 関数 

# データの読み込み

In [211]:
df = pd.read_excel("Machine_Learning_Data3000.xlsx")

In [212]:
data = df[["サウンド バイトのテキスト", "メディア タイプ"]]
data.columns = ["text", "media"]
data.head()

,text,media
0,【ミニカー予約速報】EIDOLON (アイドロン) 1/43 トヨタ GR86 RZ 202...,Image; Link
1,トヨタ ライズの買取価格は？査定額や相場がいくらか調査した！,Link
2,やっぱりレクサスはトヨタと違う！ 新型ESにあってクラウンにはない高級感とは？(GQ JAP...,Link
3,【画像】トヨタの新型プリウス、ガチのマジでかっこいい,Link
4,将来、2輪は水素直噴カワサキH2SX、4輪はトヨタ水素直噴カローラで水素生活か,No Media


In [213]:
# No‗Mediaでソート
data = data[data["media"] == "No Media"]
data = data.reset_index(drop=True)
data.head()

,text,media
0,将来、2輪は水素直噴カワサキH2SX、4輪はトヨタ水素直噴カローラで水素生活か,No Media
1,プリウスを威信材にするのがトヨタは上手かったんです。,No Media
2,鴉杜麗(あずーら)！！醒栩羅(さくら)！！琴蟇椰(こばやし)！！すまん遅れて！！ 父ちゃんが...,No Media
3,過去イチでカッコ良いトヨタ車の一つになれて良かったネ、プリウス（遅いわ）。,No Media
4,21時になりました。 今のトヨタクラウンは、『S21』、14代目。実は結構なモデルチェンジを...,No Media


In [214]:
# 統計値の算出
data.describe(include="all")

,text,media
count,608,608
unique,545,1
top,カローラ店では「エスティマL」、トヨタ店では「エスティマT」なんだー,No Media
freq,9,608


In [215]:
# 重複行の削除
data = data[data["text"].duplicated(keep='last') == False]
data = data.reset_index(drop=True)

# 統計値の算出
data.describe(include="all")

,text,media
count,545,545
unique,545,1
top,将来、2輪は水素直噴カワサキH2SX、4輪はトヨタ水素直噴カローラで水素生活か,No Media
freq,1,545


# データマイニング
全角 -> 半角変換

In [216]:
data["text"] = data["text"].map(lambda x: jaconv.z2h(x, kana=True, digit=True, ascii=True))

小文字 -> 大文字変換

In [217]:
data["text"] = data["text"].map(lambda x: x.upper())

不要文字・記号の削除(一部の処理)

In [218]:
# 【~】削除
data["text"] = data["text"]\
    .str.replace(re.compile(r"【\S+】"), "")\
    .str.replace(re.compile(r"【\S+ \S+】"), "")\
    .str.replace(re.compile(r"【\S+ \S+ \S+】"), "")

# ＃以降を削除
data["text"] = data["text"]\
    .str.replace(re.compile(r"#\S+"), "")\
    .str.replace(re.compile(r"# \S+"), "")   
# (~) 削除
data["text"] = data["text"]\
    .str.replace(re.compile(r"\(\S+\)"), "")\
    .str.replace(re.compile(r"（\S+）"), "")   

不要文字・記号の削除(全部の処理)

In [219]:
# 半角処理
data["text"] = data["text"].map(lambda x: re.sub("[!-/:-@[-`{-~]", "", x))

# 全角処理
data["text"] = data["text"].map(lambda x: re.sub("[\uFF01-\uFF0F\uFF1A-\uFF20\uFF3B-\uFF40\uFF5B-\uFF65\u3000-\u303F]", "", x))

# 空白の削除
data["text"] = data["text"].str.replace(" ", "")

# 記号の完全削除
data["text"] = data["text"].map(lambda x: re.sub("[]", "", x))

In [225]:
dl = []
for sen in data["text"]:
    for val in sen:
        bol = emoji.is_emoji(val)
        if bol:
            sen = sen.replace(val, "")
    dl.append(sen)
data["text"] = dl

In [227]:
data["text"]

0                  将来2輪は水素直噴ｶﾜｻｷH2SX4輪はﾄﾖﾀ水素直噴ｶﾛｰﾗで水素生活か
1          鴉杜麗すまん遅れて父ちゃんがKINTOで買うた直3のﾔﾘｽで急いで行くでﾄﾖﾀ共販にﾆｶｯ
2                        過去ｲﾁでｶｯｺ良いﾄﾖﾀ車の一つになれて良かったﾈﾌﾟﾘｳｽ
3      30系ﾌﾟﾘｳｽはいい加減使用禁止にした方がいいTOYOTAは自分が造った車でどれほど多くの...
4                               ﾌﾟﾘｳｽのお尻好きだな最近のﾄﾖﾀ全部これだわ
5      ﾌｪﾗｰﾘじゃないんだからあそこまでﾌﾛﾝﾄを寝かせる必要はないだろうそもそもﾌｧﾐﾘｶｰだ...
6                                  ｸﾙﾏの名前ﾄﾖﾀｶﾑﾘは冠をもじったもの
7      ｶﾛｰﾗGRFOURなんか壊れないRS3なのに買いたいと全く思えないのよなﾄﾖﾀに伝説的なｽ...
8      おはよう︎昨日寝れなくて1時間しか寝てないそして今日運転やばいﾄﾖﾀのｸﾗｳﾝがいて事故りか...
9                      所謂USDM党で新型を購入された方へｺﾝﾊﾞｰｼﾞｮﾝにする予定は
10     ﾀﾞﾝﾅのことだから私がとやかく言えないんだけど…ﾄﾖﾀの新型○○とかのCM見ると新型ばっか...
11         新型ﾌﾟﾘｳｽかっこよい…あとﾊｲﾊﾟﾜｰですごい…しかしなぜ窓を小さくしたんだﾄﾖﾀさん
12                新型ﾌﾟﾘｳｽ発表いつまでﾊｲﾌﾞﾘｯﾄﾞを作り続けるんだへのﾄﾖﾀの答えは
13                                  ﾄﾖﾀﾌﾟﾘｳｽ式ｼﾌﾄｺﾞﾘ押しやん…
14     2712011年に初音ﾐｸがｱﾒﾘｶでCMに起用され話題となったﾄﾖﾀの自動車は→ｶﾛｰﾗ※...
15     EVはｴｺで万能みたいな事するからﾄﾖﾀが怒ってｴｺでﾀﾞｻいﾌﾟﾘｳｽを変えてしまった感じ...
16     はいだしょうこの81ｼｴﾝﾀまでのｷﾞｬｸﾞが通じるﾄﾖﾀ好きは中おらん負ったとしても頭のﾈ...
17     今月20日から約1か月間なんば駅前の大型ﾓﾆﾀｰ